<center> <img src="https://storage.cloud.google.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="300"/> </center>

# <center>Getting Started with Hosted Phoenix

This guide demonstrates how to use hosted phoenix, our solution for AI developers to trace and evaluate their LLM applications without setting up infrastructure. Read more about Phoenix [here](https://docs.arize.com/phoenix/).

ℹ️ This notebook requires an OpenAI API key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies. It does NOT require installing arize-phoenix to log traces, as we are compliant with OpenTelemetry tracing.

In [1]:
# !pip install -q "arize-phoenix>=4.29.0" openinference-instrumentation-openai openai

In [2]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
import os
from getpass import getpass

PHOENIX_API_KEY = "usertest:usertest"
# if not (PHOENIX_API_KEY := os.getenv("PHOENIX_API_KEY")):
#     PHOENIX_API_KEY = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_API_KEY"] = PHOENIX_API_KEY

## Step 2: Setup Tracing
Let's send a trace to Hosted Phoenix!

In [4]:
!pip freeze | grep phoenix

-e git+ssh://git@github.com/Arize-ai/phoenix.git@8c24822d8445cb3877bb95277524ea8d06bb7b07#egg=arize_phoenix
arize-phoenix-evals==0.15.0
-e git+ssh://git@github.com/Arize-ai/phoenix.git@8c24822d8445cb3877bb95277524ea8d06bb7b07#egg=arize_phoenix_otel&subdirectory=packages/phoenix-otel


In [5]:
from openinference.instrumentation.openai import OpenAIInstrumentor

from phoenix.otel import register

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ['OTEL_EXPORTER_OTLP_TRACES_INSECURE']='true' 
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://localhost:4041"

# Setup OTEL tracing for hosted Phoenix. The register function will automatically detect the endpoint and headers from your environment variables.
tracer_provider = register()

# Turn on instrumentation for OpenAI
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

INFO:phoenix.config:📋 Ensuring phoenix working directory: /Users/kikocastillo/.phoenix
INFO:phoenix.inferences.inferences:Dataset: phoenix_inferences_cddd9534-1b11-4de6-8058-ce9ee12bf70e initialized


HELLO KIKO
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'api_key': 'usertest:usertest', 'authorization': 'Bearer usertest:usertest', 'user-agent': 'OTel-OTLP-Exporter-Python/1.26.0'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Send query to Hosted Phoenix


In [6]:
import openai

openai_client = openai.OpenAI()
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Write a haiku."}],
    max_tokens=20,
)
print(response.choices[0].message.content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Nature whispers soft
Leaves rustle in the gentle breeze
Peaceful serenade


## Step 3: Access Phoenix Instance

From here, you can access your Phoenix instance to power evaluations, experiments, upload datasets, etc.


In [7]:
from datetime import datetime, timedelta

import phoenix as px

# Initiate Phoenix client
px_client = px.Client()

# Get spans from the last 7 days only
start = datetime.now() - timedelta(days=7)

# Get spans to exclude the last 24 hours
end = datetime.now() - timedelta(days=0)

phoenix_df = px_client.query_spans(start_time=start, end_time=end)
print(phoenix_df.head())

INFO:httpx:HTTP Request: GET http://localhost:4041/arize_phoenix_version "HTTP/1.1 200 OK"
/Users/kikocastillo/Documents/Arize/phoenix/src/phoenix/utilities/client.py:25: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(
INFO:httpx:HTTP Request: POST http://localhost:4041/v1/spans?project_name=default&project-name=default "HTTP/1.1 200 OK"
/Users/kikocastillo/Documents/Arize/phoenix/src/phoenix/utilities/client.py:45: UserWarning: The Phoenix server (4.32.0) and client (4.35.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


OSError: Invalid IPC stream: negative continuation token

Use our latest and greatest features around datasets on the latest Phoenix version

In [ ]:
import pandas as pd

df = pd.DataFrame(
    [
        {
            "question": "What is Paul Graham known for?",
            "answer": "Co-founding Y Combinator and writing on startups and techology.",
            "metadata": {"topic": "tech"},
        }
    ]
)
phoenix_client = px.Client()
dataset = phoenix_client.upload_dataset(
    dataframe=df,
    dataset_name="test-dataset-2",
    input_keys=["question"],
    output_keys=["answer"],
    metadata_keys=["metadata"],
)